## Import

In [23]:
import json
from tqdm import tqdm

In [24]:
import openai
openai.api_key = 'sk-xxxxxx'

In [25]:
instruction = '''以下是关于医学知识的单项选择题，请根据题目输出唯一的答案选项\n'''
instruction

'以下是关于医学知识的单项选择题，请根据题目输出唯一的答案选项\n'

In [26]:
path_file_data = '../data/MedQA/Mainland/test.jsonl'

## Data

In [27]:
list_dict_test = []
with open(path_file_data, 'r', encoding="utf-8") as f:
    for idx, line in enumerate(f):
        data = json.loads(line)
        data['ID'] = idx
        data['A'], data['B'], data['C'], data['D'], data['E'] = data['options']['A'], data['options']['B'], data['options']['C'], data['options']['D'], data['options']['E']
        del data['options']
        list_dict_test.append(data)

In [28]:
list_dict_test[0]

{'question': '经调查证实出现医院感染流行时，医院应报告当地卫生行政部门的时间是（\u3000\u3000）。',
 'answer': '24小时内',
 'meta_info': '卫生法规',
 'answer_idx': 'E',
 'ID': 0,
 'A': '2小时',
 'B': '4小时内',
 'C': '8小时内',
 'D': '12小时内',
 'E': '24小时内'}

In [17]:
for idx, dict_test in enumerate(tqdm(list_dict_test)):
    question, answer = dict_test['question'], dict_test['answer']
    a, b, c, d, e = dict_test['A'], dict_test['B'], dict_test['C'], dict_test['D'], dict_test['E']
    question = question.replace("（ ）。", "")
    input = instruction + f"问题：{question}: (A){a}, (B){b}, (C){c}, (D){d}, (E){e}\n" + "答案："
    dict_test['Input'] = input
    # print(input, '\t' ,answer)

100%|██████████| 3426/3426 [00:00<00:00, 418843.58it/s]


## Test

In [18]:
# config of model
model = "gpt-4-0613"
cost_input_1k = 0.03
cost_output_1k = 0.06
# "gpt-3.5-turbo-0613"
# cost_input_1k = 0.0015
# cost_output_1k = 0.002
# 'gpt-3.5-0613'
temperature = 0
top_p = 1
frequency_penalty = 0
presence_penalty = 0
prompt_system = """You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: max_thread21-09
Current date: max_thread23-07-01"""

In [19]:
def get_response_idx(input):
    response = openai.ChatCompletion.create(
        model=model, 
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        # max_tokens = 1,
        messages=[
            # {"role": "system", "content": prompt_system}, 
            {"role": "user", "content": input}
        ],
        timeout=60,
    )
    print(f'Response: {response["choices"][0]["message"]["content"]}', )
    print(f"Finish_reason: {response['choices'][0]['finish_reason']}")
    print(response['usage']._previous)
    return response

In [21]:
idx = 99
input = list_dict_test[idx]['Input']
print(input)
response = get_response_idx(input)
print(f"Ground-truth: {list_dict_test[idx]['answer']}")

以下是关于医学知识的单项选择题，请根据题目输出唯一的答案选项
问题：下述有关主动脉粥样硬化的叙述中，哪一项错误？（　　）: (A)病变多位于主动脉后壁和各分支开口处, (B)可引起夹层动脉瘤, (C)可引起主动脉瓣病变, (D)胸主动脉病变最重, (E)可继发溃疡、钙化、出血、血栓形成
答案：
Response: (D)胸主动脉病变最重
Finish_reason: stop
{'prompt_tokens': 186, 'completion_tokens': 15, 'total_tokens': 201}
Ground-truth: 胸主动脉病变最重


## Task

In [ ]:
name_task = "Prompt-gpt4"

In [ ]:
with open(f'../result/{name_task}.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(list_dict_test, indent=2, ensure_ascii=False))

In [ ]:
list_dict_test[0]

## Run

In [ ]:
name_task

In [ ]:
!python API_multithread.py "{openai.api_key}" "{name_task}"

## Performance

### Read

In [ ]:
with open(f'../result/{name_task}.json', 'r', encoding="utf-8") as f:
    list_dict_test = json.load(f)

In [ ]:
len(list_dict_test), len([dict_test for dict_test in list_dict_test if dict_test.get('usage')])

In [ ]:
list_dict_test[50]

### Supplement

In [ ]:
flag_failed = True

In [ ]:
list_dict_result_failed = [ dict_test for dict_test in list_dict_test if not dict_test.get('usage') ]
len(list_dict_result_failed)

In [ ]:
if flag_failed:
    for dict_test in tqdm(list_dict_result_failed):
        input = dict_test["Input"]
        id = dict_test["ID"]
        try:
            response = openai.ChatCompletion.create(
                model=model,
                temperature=temperature,
                top_p=top_p,
                frequency_penalty=frequency_penalty,
                presence_penalty=presence_penalty,
                # max_tokens = 1,
                messages=[
                    # {"role": "system", "content": prompt_system},
                    {"role": "user", "content": input},
                ],
                timeout=60,
            )
            result = response["choices"][0]["message"]["content"]
            dict_test["Result"] = result
            # usage
            dict_test["usage"] = response["usage"]._previous
            # finish reason
            dict_test["finish_reason"] = response["choices"][0]["finish_reason"]
            # print(input, result, sep='\n')
        except:
            print(f"Wrong: {id}\n")
    print('Supplement failed done.')

In [ ]:
with open(f'../result/{name_task}.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(list_dict_test, indent=2, ensure_ascii=False))

### Prediction

In [ ]:
def get_prediction(result_pred, list_option_name):
    list_str_split = ['最终答案是','最终答案为','最可能的选项是','最可能的诊断是','正确的选项是', '正确选项是', '正确选项为','最可能的答案是','最有可能的答案是']
    for str_split in list_str_split:
        if len(result_pred.split(str_split))>1:
            result_pred = result_pred.split(str_split)[-1]
            result_pred = result_pred.split('。')[0]
            break
    list_option_idx = [ chr(ord('A') + idx) for idx in range(len(list_option_name))]
    list_option_idx_name = [ [option_idx, option_name] for option_idx, option_name in zip(list_option_idx, list_option_name)]
    list_option_idx_name = sorted(list_option_idx_name, key=lambda x: len(x[1]), reverse=True)
    list_option_match = []
    # search option_idx and option_name
    for option_idx, option_name in list_option_idx_name:
        if option_idx in result_pred or option_name in result_pred:
            list_option_match.append(option_idx)
            if option_name in result_pred:
                result_pred = result_pred.replace(option_name, '')
    if len(list_option_match)==1:
        return list_option_match[0]
    elif len(list_option_match)>1:
        return None
    else:
        return None

In [ ]:
list_dict_result_correct, list_dict_result_wrong, list_dict_result_none = [], [], []
for idx, dict_test in enumerate(tqdm(list_dict_test)):
    list_option_name = [dict_test['A'], dict_test['B'], dict_test['C'], dict_test['D'], dict_test['E']]
    dict_test['Prediction'] = get_prediction(dict_test['Result'], list_option_name)
    if not dict_test['Prediction']:
        list_dict_result_none.append(dict_test)
    elif dict_test['answer'] == dict_test['Prediction']:
        list_dict_result_correct.append(dict_test)
    else:
        list_dict_result_wrong.append(dict_test)

In [ ]:
count_correct = len(list_dict_result_correct)
count_wrong = len(list_dict_result_wrong)
count_none = len(list_dict_result_none)
acc = round(len(list_dict_result_correct) / len(list_dict_test)*100, 2)
print(f"Accuracy is {acc}% with {count_correct} correct, {count_wrong} wrong, and {count_none} none")

In [ ]:
with open(f'../result/{name_task}.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(list_dict_test, indent=4, ensure_ascii=False))

### Accuracy for different types of questions

In [ ]:
with open(f'../result/{name_task}.json', 'r', encoding="utf-8") as f:
    list_dict_test_sample = json.load(f)
with open('../data/list_test_knowledge_idx.txt', 'r') as f:
    list_test_knowledge_idx = f.readlines()
with open('../data/list_test_example_idx.txt', 'r') as f:
    list_test_example_idx = f.readlines()

In [ ]:
list_test_knowledge_idx = [ int(idx.strip()) for idx in list_test_knowledge_idx if idx.strip() ]
list_test_example_idx = [ int(idx.strip()) for idx in list_test_example_idx if idx.strip() ]
list_dict_test_knowledge = [ list_dict_test_sample[idx] for idx in list_test_knowledge_idx ]
list_dict_test_example = [ list_dict_test_sample[idx] for idx in list_test_example_idx ]
print(len(list_dict_test_sample), len(list_dict_test_knowledge), len(list_dict_test_example))

In [ ]:
def cal_performance(list_dict_test_sample):
    list_dict_result_correct, list_dict_result_wrong, list_dict_result_none = [], [], []
    for idx, dict_test in enumerate(list_dict_test_sample):
        if not dict_test['Prediction']:
            list_dict_result_none.append(dict_test)
        elif dict_test['answer'] == dict_test['Prediction']:
            list_dict_result_correct.append(dict_test)
        else:
            list_dict_result_wrong.append(dict_test)
    count_correct = len(list_dict_result_correct)
    count_wrong = len(list_dict_result_wrong)
    count_none = len(list_dict_result_none)
    acc = len(list_dict_result_correct) / len(list_dict_test_sample)
    print(f"Accuracy is {round(acc*100, 2)}% with {count_none} none, {count_correct} correct, and {count_wrong} wrong， total {len(list_dict_test_sample)}")
    return round(acc, 4), count_none, count_correct, count_wrong

In [ ]:
acc_MK, count_none_MK, _, _ = cal_performance(list_dict_test_knowledge)
acc_CA, count_none_CA, _, _ = cal_performance(list_dict_test_example)
acc_all, count_none_all, _, _ = cal_performance(list_dict_test_sample)

In [ ]:
print(count_none_MK, acc_MK, count_none_CA, acc_CA, count_none_all, acc_all, sep='\t')

## End

In [ ]:
def cal_cost(token_input, token_output, cost_input_1k, cost_output_1k):
    cost_input = token_input * cost_input_1k / 1000 * 7.24
    cost_output = token_output * cost_output_1k / 1000 * 7.24
    print(f"cost_input: {cost_input:.2f} rmb")
    print(f"cost_output: {cost_output:.2f} rmb")
    print(f"All cost: {(cost_input+cost_output):.2f} rmb")

In [ ]:
token_input = sum([dict_test["usage"]["prompt_tokens"] for dict_test in list_dict_test_sample])
token_output = sum(
    [dict_test["usage"]["completion_tokens"] for dict_test in list_dict_test_sample]
)
cal_cost(token_input, token_output, cost_input_1k, cost_output_1k)